# Head Direction Analyses

Analysis of head direction.

In [1]:
%config Completer.use_jedi = False

In [2]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from pynwb import NWBHDF5IO

In [3]:
import seaborn as sns
sns.set_context('talk')

## Settings

In [4]:
# Set which task to analyze
task = 'THF'

In [5]:
# Define the base data folder
data_folder = Path('/Users/tom/Documents/Data/JacobsLab/TH/' + task)

In [6]:
# Define subject information
subj = 'wv001'
session = 0

## Load Data

In [7]:
# Define expected NWB file name & full path
file_name = '_'.join([subj, 'session_' + str(session)]) + '.nwb'
full_path = data_folder / 'NWB' / file_name

In [8]:
# Load NWB file
io = NWBHDF5IO(str(full_path), 'r')
nwbfile = io.read()